In [4]:

from torch import nn, optim, utils
import numpy as np
import os
import time
import dill
import json
import random
import pathlib
import warnings
from tqdm import tqdm
import visualization
import evaluation
import matplotlib.pyplot as plt
from model.trajectron import Trajectron
from model.model_registrar import ModelRegistrar
from model.model_utils import cyclical_lr
from model.dataset import EnvironmentDataset, collate
from tensorboardX import SummaryWriter


In [5]:
!ls /data/jedrzej/argo2city/processed/val/

argo2city_austin_1.pkl	      argo2city_pittsburgh_2.pkl
argo2city_austin_2.pkl	      argo2city_pittsburgh_3.pkl
argo2city_austin_3.pkl	      argo2city_pittsburgh_4.pkl
argo2city_austin_4.pkl	      argo2city_pittsburgh_5.pkl
argo2city_austin_5.pkl	      argo2city_pittsburgh_full.pkl
argo2city_austin_full.pkl     argo2city_washington-dc_1.pkl
argo2city_dearborn_1.pkl      argo2city_washington-dc_2.pkl
argo2city_dearborn_2.pkl      argo2city_washington-dc_3.pkl
argo2city_dearborn_3.pkl      argo2city_washington-dc_4.pkl
argo2city_dearborn_4.pkl      argo2city_washington-dc_5.pkl
argo2city_dearborn_5.pkl      argo2city_washington-dc_full.pkl
argo2city_dearborn_full.pkl   austin_full.pkl
argo2city_miami_1.pkl	      austin_valenv.pkl
argo2city_miami_2.pkl	      dearborn_full.pkl
argo2city_miami_3.pkl	      dearborn_valenv.pkl
argo2city_miami_4.pkl	      miami_full.pkl
argo2city_miami_5.pkl	      miami_valenv.pkl
argo2city_miami_full.pkl      palo-alto_full.pkl
argo2city_palo-alto_1.pkl     p

In [41]:
data_dir = '/data/jedrzej/argo2city/processed'
eval_data_dict = 'val/argo2city_austin_{}.pkl'

conf = '../config/argo2.json'
with open(conf, 'r', encoding='utf-8') as conf_json:
    hyperparams = json.load(conf_json)

In [52]:
from tqdm import tqdm
eval_scenes = []

for i in tqdm(range(1,6)):
    
    eval_scenes_sample_probs = None

    eval_data_path = os.path.join(data_dir, eval_data_dict.format(i))
    with open(eval_data_path, 'rb') as f:
        eval_env = dill.load(f, encoding='latin1')
        eval_scenes+= eval_env.scenes

eval_env.scenes = eval_scenes
eval_scenes_sample_probs = None


100%|██████████| 5/5 [00:09<00:00,  1.90s/it]


In [53]:
len(eval_scenes)

5185

In [59]:
scene = eval_scenes[0]

In [60]:
dir(scene)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'add_robot_from_nodes',
 'aug_func',
 'augment',
 'calculate_scene_graph',
 'description',
 'dt',
 'duration',
 'frequency_multiplier',
 'get_clipped_input_dict',
 'get_node_by_id',
 'get_nodes_clipped_at_time',
 'get_scene_graph',
 'map',
 'name',
 'nodes',
 'non_aug_scene',
 'present_nodes',
 'robot',
 'sample_timesteps',
 'temporal_scene_graph',
 'timesteps']

In [61]:
scene.name

'61d3a2e0-54c9-4551-a93a-9a5f6b9a8491'

In [39]:
hyperparams['prediction_horizon']

59

In [32]:
for node_type_data_set in eval_dataset:
    print(node_type_data_set)


In [54]:
eval_dataset = EnvironmentDataset(eval_env,
                                  hyperparams['state'],
                                  hyperparams['pred_state'],
                                  scene_freq_mult=False,
                                  node_freq_mult=False,
                                  hyperparams=hyperparams,
                                  min_history_timesteps=hyperparams['minimum_history_length'],
                                  min_future_timesteps=hyperparams['prediction_horizon'],
                                  return_robot=True)
eval_data_loader = dict()

for node_type_data_set in eval_dataset:
    print(node_type_data_set.node_type,len(node_type_data_set))
    #print(node_type_data_set[0])

    if len(node_type_data_set) == 0:
        continue

    node_type_dataloader = utils.data.DataLoader(node_type_data_set,
                                                 collate_fn=collate,
                                                 pin_memory=True,
                                                 batch_size=256,
                                                 shuffle=True,
                                                 num_workers=10)
    eval_data_loader[node_type_data_set.node_type] = node_type_dataloader

print(f"Loaded evaluation data from {eval_data_path}")

PEDESTRIAN 9865
VEHICLE 292055
Loaded evaluation data from /data/jedrzej/argo2city/processed/val/argo2city_austin_1.pkl


In [58]:
model_dir = '../experiments/argo2/models/models_austin_test_small'
device = 'cuda'

log_writer = SummaryWriter(log_dir=model_dir)

model_registrar = ModelRegistrar(model_dir, device) #args.device

model_registrar.load_models(iter_num=17)

eval_trajectron = Trajectron(model_registrar,
                                 hyperparams,
                                 log_writer,
                                 device)
eval_trajectron.set_environment(eval_env)

PermissionError: [Errno 13] Permission denied: '../experiments/argo2/models/models_austin_test_small/events.out.tfevents.1645262320.south'

In [57]:
max_hl = hyperparams['maximum_history_length']
ph = hyperparams['prediction_horizon']
model_registrar.to(args.eval_device)
with torch.no_grad():
    # Predict batch timesteps for evaluation dataset evaluation
    eval_batch_errors = []
    for scene in tqdm(eval_scenes, desc='Sample Evaluation', ncols=80):
        timesteps = scene.sample_timesteps(args.eval_batch_size)

        predictions = eval_trajectron.predict(scene,
                                              timesteps,
                                              ph,
                                              num_samples=50,
                                              min_future_timesteps=ph,
                                              full_dist=False)

        eval_batch_errors.append(evaluation.compute_batch_statistics(predictions,
                                                                     scene.dt,
                                                                     max_hl=max_hl,
                                                                     ph=ph,
                                                                     node_type_enum=eval_env.NodeType,
                                                                     map=scene.map))

        print(eval_batch_errors)
        break

NameError: name 'model_registrar' is not defined